# ETL

## Library

In [2]:
import os 
import pandas as pd
import boto3
import yaml
import requests
from INEGIpy import Indicadores

## Tokens

In [3]:
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

# Conexión a APIs
banxico_token = config["API"]["BANXICO_TOKEN"]
inegi_token = config["API"]["INEGI_TOKEN"]

## Extract 

In [54]:

def extract_info_banxico(series_id: str) -> pd.DataFrame:

    url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{series_id}/datos/2000-01-01/2018-01-08"

    data = requests.get(url, headers={"Bmx-Token": banxico_token}).json()


    series = data['bmx']['series'][0]  # Assuming one series
    data = pd.DataFrame(series['datos'])

    return data


def extract_info_inegi() -> pd.DataFrame:
    inegi = Indicadores(inegi_token)

    data = inegi.obtener_df(indicadores = ["216377"], 
                      nombres = ['Inflacion'], 
                      inicio = '2000', 
                      fin = '2018')
    
    return data


series_id = ["SF63528","SF61745"]

# Extraccion de tasa de interes, tipo de cambio e inflacion
tasa = extract_info_banxico(series_id[0])
tipo_cambio = extract_info_banxico(series_id[1])
inflacion = extract_info_inegi()




    

/opt/conda/envs/arquitectura/lib/python3.11/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.set_index(pd.to_datetime(df.fechas),inplace=True, drop=True)


## Transform

In [55]:
def clean_data_tasa(data: pd.DataFrame) -> pd.DataFrame:
    data["date"] = pd.to_datetime(data["fecha"], format="%d/%m/%Y")
    data["tasa_de_interes"] = pd.to_numeric(data["dato"])
    data.drop(columns=["fecha", "dato"], inplace=True)
    return data

def clean_data_cambio(data: pd.DataFrame) -> pd.DataFrame:
    data["date"] = pd.to_datetime(data["fecha"], format="%d/%m/%Y")
    data["tipo_de_cambio"] = pd.to_numeric(data["dato"])
    data.drop(columns=["fecha", "dato"], inplace=True)
    return data

def clean_data_inflacion(data: pd.DataFrame) -> pd.DataFrame:
    data["date"] = pd.to_datetime(data.index, format="%d/%m/%Y")
    data["inflacion"] = pd.to_numeric(data["Inflacion"])
    data.drop(columns=["Inflacion"], inplace=True)
    data.reset_index(drop=True, inplace=True)
    return data

tasa = clean_data_tasa(tasa)
tipo_cambio = clean_data_cambio(tipo_cambio)
inflacion = clean_data_inflacion(inflacion)


## Load

In [56]:
tasa.to_csv("data/tasa.csv", index=False)
tipo_cambio.to_csv("data/tipo_cambio.csv", index=False)
inflacion.to_csv("data/inflacion.csv", index=False)


In [57]:
# Subir archivos a S3
session = boto3.Session(profile_name='arquitectura')
s3 = session.client('s3')
BUCKET_NAME = "itam-analytics-mau"

In [58]:
# upload de archivos

s3.upload_file(Filename="data/inflacion.csv", Bucket=BUCKET_NAME, Key="econ/raw/inflacion/inflacion.csv")
s3.upload_file(Filename="data/tasa.csv", Bucket=BUCKET_NAME, Key="econ/raw/tasa/tasa.csv")
s3.upload_file(Filename="data/tipo_cambio.csv", Bucket=BUCKET_NAME, Key="econ/raw/tipo_cambio/tipo_cambio.csv")